## QE methods and QE_utils

In this tutorial, we will explore various methods needed to handle Quantum Espresso (QE) calculations - to run them, prepare input, and extract output. All that will be done with the help of the **QE_methods** and **QE_utils** modules, which contains the following functions:

**QE_methods**
* cryst2cart(a1,a2,a3,r)
* [Topic 2](#topic-2) read_qe_schema(filename, verbose=0)
* [Topic 3](#topic-3) read_qe_index(filename, orb_list, verbose=0)
* [Topic 4](#topic-4) read_qe_wfc_info(filename, verbose=0)
* [Topic 9](#topic-9) read_qe_wfc_grid(filename, verbose=0)
* [Topic 5](#topic-5) read_qe_wfc(filename, orb_list, verbose=0)
* read_md_data(filename)
* read_md_data_xyz(filename, PT, dt)
* read_md_data_xyz2(filename, PT)   
* read_md_data_cell(filename)
* out2inp(out_filename,templ_filename,wd,prefix,t0,tmax,dt)
* out2pdb(out_filename,T,dt,pdb_prefix)
* out2xyz(out_filename,T,dt,xyz_filename)
* xyz2inp(out_filename,templ_filename,wd,prefix,t0,tmax,dt)
* get_QE_normal_modes(filename, verbosity=0)
* [Topic 1](#topic-1) run_qe(params, t, dirname0, dirname1)                         
* read_info(params)
* read_all(params)
* read_wfc_grid(params)

**QE_utils**
* get_value(params,key,default,typ)
* split_orbitals_energies(C, E)
* [Topic 7](#topic-7) merge_orbitals(Ca, Cb)
* post_process(coeff, ene, issoc)
* [Topic 6](#topic-6) orthogonalize_orbitals(C)
* [Topic 8](#topic-8) orthogonalize_orbitals2(Ca,Cb)

In [1]:
import os
import sys
import math
import copy

if sys.platform=="cygwin":
    from cyglibra_core import *
elif sys.platform=="linux" or sys.platform=="linux2":
    from liblibra_core import *
#from libra_py import *
from libra_py import units
from libra_py import QE_methods
from libra_py import QE_utils
from libra_py import scan

from libra_py import hpc_utils
from libra_py import data_read
from libra_py import data_outs
from libra_py import data_conv
from libra_py.workflows.nbra import step2


import py3Dmol   # molecular visualization
import matplotlib.pyplot as plt   # plots
%matplotlib inline 

plt.rc('axes', titlesize=24)      # fontsize of the axes title
plt.rc('axes', labelsize=20)      # fontsize of the x and y labels
plt.rc('legend', fontsize=20)     # legend fontsize
plt.rc('xtick', labelsize=16)    # fontsize of the tick labels
plt.rc('ytick', labelsize=16)    # fontsize of the tick labels

plt.rc('figure.subplot', left=0.2)
plt.rc('figure.subplot', right=0.95)
plt.rc('figure.subplot', bottom=0.13)
plt.rc('figure.subplot', top=0.88)

colors = {}

colors.update({"11": "#8b1a0e"})  # red       
colors.update({"12": "#FF4500"})  # orangered 
colors.update({"13": "#B22222"})  # firebrick 
colors.update({"14": "#DC143C"})  # crimson   

colors.update({"21": "#5e9c36"})  # green
colors.update({"22": "#006400"})  # darkgreen  
colors.update({"23": "#228B22"})  # forestgreen
colors.update({"24": "#808000"})  # olive      

colors.update({"31": "#8A2BE2"})  # blueviolet
colors.update({"32": "#00008B"})  # darkblue  

colors.update({"41": "#2F4F4F"})  # darkslategray

clrs_index = ["11", "21", "31", "41", "12", "22", "32", "13","23", "14", "24"]

/home/alexey/miniconda2/envs/py37/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: to-Python converter for std::vector<std::complex<double>, std::allocator<std::complex<double> > > already registered; second conversion method ignored.
  return f(*args, **kwds)
/home/alexey/miniconda2/envs/py37/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: to-Python converter for std::vector<std::vector<int, std::allocator<int> >, std::allocator<std::vector<int, std::allocator<int> > > > already registered; second conversion method ignored.
  return f(*args, **kwds)
/home/alexey/miniconda2/envs/py37/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: to-Python converter for boost::python::detail::container_element<std::vector<std::vector<int, std::allocator<int> >, std::allocator<std::vector<int, std::allocator<int> > > >, unsigned long, boost::python::detail::final_vector_derived_policies<std::vector<std::vector<int, std::allocator<int> >, std::allocator<std::vector<int, s

First, lets prepare the working directories and run simple SCF calculations to generate the output files

In [ ]:
# Setup the calculations
params = {}

# I run the calculations on laptop, so no BATCH system
params["BATCH_SYSTEM"] = None

# The number of processors to use
params["NP"] = 1

# The QE executable
params["EXE"] = "/mnt/c/cygwin/home/Alexey-user/Soft/espresso/bin/pw.x"

# The executable to generate the wavefunction files
params["EXE_EXPORT"] = "/mnt/c/cygwin/home/Alexey-user/Soft/espresso/bin/pw_export.x"

# The type of the calculations to be performed - in this case only a single SCF with spin-polarization
params["nac_method"] = 1
    
# The prefix of the input file
params["prefix0"] = "x0.scf" 

# Working directory - where all stuff happen
params["wd"] = os.getcwd()+"/wd"


# Remove the previous results and temporary working directory from the previous runs
os.system(F"rm -r {params['wd']}")
os.system(F"mkdir {params['wd']}")

# Copy the input files into the working directory
# also, notice how the SCF input file name has been changed
os.system(F"cp x0.scf.in {params['wd']}/x0.scf.0.in")
os.system(F"cp x0.exp.in {params['wd']}")
os.system(F"cp Li.pbe-sl-rrkjus_psl.1.0.0.UPF {params['wd']}")
os.system(F"cp H.pbe-rrkjus_psl.1.0.0.UPF {params['wd']}")


<a name="topic-1"></a>
### 1. run_qe(params, t, dirname0, dirname1)


Use it to actually run the calculations

Comment this out if you have already done the calculations

In [ ]:
os.chdir("wd")
QE_methods.run_qe(params, 0, "res", "res2")
os.chdir("../")

<a name="topic-2"></a>
### 2. read_qe_schema(filename, verbose=0)

Can be used to read the information about the completed run

In [23]:
pwd

'/mnt/c/cygwin/home/Alexey-user/Programming/Project_libra/notebooks/Example14_QE_methods'

In [24]:
info = QE_methods.read_qe_schema("wd/res/x0.save/data-file-schema.xml", verbose=0)
print(info)

{'conv': 8, 'etot': -7.9497569277338, 'nbnd': 20, 'nelec': 4, 'efermi': -0.12376709627891, 'alat': 37.79452265771, 'nat': 2, 'coords': <liblibra_core.MATRIX object at 0x7fae6817b228>, 'atom_labels': ['Li', 'H'], 'forces': <liblibra_core.MATRIX object at 0x7fae6817b500>}


In [25]:
nat = info["nat"]
R, F = info["coords"], info["forces"]

for at in range(nat):
    print(F"Atom {at} \t {info['atom_labels'][at]} \t\
          x={R.get(3*at+0):.5f}, y={R.get(3*at+1):.5f}, z={R.get(3*at+2):.5f}\
          fx={F.get(3*at+0):.5f}, fy={F.get(3*at+1):.5f}, fz={F.get(3*at+2):.5f}")

Atom 0 	 Li 	          x=0.00000, y=0.00000, z=0.00000          fx=0.00000, fy=0.00000, fz=0.00000
Atom 1 	 H 	          x=2.83459, y=0.00000, z=0.00000          fx=0.00000, fy=0.00000, fz=0.00000


<a name="topic-3"></a>
### 3. read_qe_index(filename, orb_list, verbose=0)

Is analogous to **read_qe_schema** in many regards, it just extracts a bit different info, including orbital energies.

One would also need to specify which energy levels we want to extract, so one would need that info beforehands.

In this example, we have just 4 electrons, so:

1 - HOMO-1
2 - HOMO 
3 - LUMO
4 - LUMO+1

Lets try just the 4 orbitals

In [26]:
info2, all_e = QE_methods.read_qe_index("wd/res/x0.export/index.xml", [1,2,3,4], verbose=1)

path= Root
 nspin =  2  nk =  2  nbnd =  10  efermi =  -0.12376709627891  alat =  37.79452265771  omega =  8637.868266355206  tpiba =  0.1662459230953623  tpiba2 =  0.1662459230953623
 Direct lattice vectors: 
 a1 =  37.79452265771286 0.0 0.0
 a2 =  0.0 15.11780906308515 0.0
 a3 =  0.0 0.0 15.11780906308515
 Reciprocal lattice vectors: 
 b1 =  0.1662459230953497 0.0 0.0
 b2 =  0.0 0.4156148077383742 0.0
 b3 =  0.0 0.0 0.4156148077383742
 K points: 
0  weight =  1.0  k =  0.0 0.0 0.0
1  weight =  1.0  k =  0.0 0.0 0.0
 Energies of the active orbitals for all k-points: 
ik =  0

ik =  1



In [27]:
print( info2)

{'nspin': 2, 'nk': 2, 'nbnd': 10, 'efermi': -0.12376709627891, 'alat': 37.79452265771, 'omega': 8637.868266355206, 'tpiba': 0.1662459230953623, 'tpiba2': 0.02763770694582913, 'a1': <liblibra_core.VECTOR object at 0x7fae681b5df8>, 'a2': <liblibra_core.VECTOR object at 0x7fae68178500>, 'a3': <liblibra_core.VECTOR object at 0x7fae681789d0>, 'b1': <liblibra_core.VECTOR object at 0x7fae68178378>, 'b2': <liblibra_core.VECTOR object at 0x7fae68178fb8>, 'b3': <liblibra_core.VECTOR object at 0x7fae681787d8>, 'weights': [1.0, 1.0], 'k': [<liblibra_core.VECTOR object at 0x7fae68178ea0>, <liblibra_core.VECTOR object at 0x7fae68178ca8>]}


In [28]:
print(all_e)

[<liblibra_core.CMATRIX object at 0x7fae68178dc0>, <liblibra_core.CMATRIX object at 0x7fae68178df8>]


In [29]:
e_alp = all_e[0]
e_bet = all_e[1]

In [30]:
for i in range(4):
    print(F"E_{i}^alpha = {e_alp.get(i,i).real:12.8f} \t E_{i}^beta = {e_bet.get(i,i).real:12.8f}")

E_0^alpha =  -1.86244053 	 E_0^beta =  -1.86219594
E_1^alpha =  -0.16462949 	 E_1^beta =  -0.16475681
E_2^alpha =  -0.08691250 	 E_2^beta =  -0.08650036
E_3^alpha =  -0.04025010 	 E_3^beta =  -0.04013592


<a name="topic-4"></a>
### 4. read_qe_wfc_info(filename, verbose=0)

Can be used to extract some descriptors of the wavefunctions produced

In [31]:
wfc_info1 = QE_methods.read_qe_wfc_info("wd/res/x0.export/wfc.1", verbose=1)
wfc_info2 = QE_methods.read_qe_wfc_info("wd/res/x0.export/wfc.2", verbose=1)

path= Kpoint.1
 ngw =  36889  igwx =  36889  nbnd =  10  nspin =  2  gamma_only =  F  ik =  1  nk =  2
path= Kpoint.2
 ngw =  36889  igwx =  36889  nbnd =  10  nspin =  2  gamma_only =  F  ik =  2  nk =  2


In [32]:
print(wfc_info1)
print(wfc_info2)

{'ngw': 36889, 'igwx': 36889, 'nbnd': 10, 'nspin': 2, 'gamma_only': 'F', 'ik': 1, 'nk': 2}
{'ngw': 36889, 'igwx': 36889, 'nbnd': 10, 'nspin': 2, 'gamma_only': 'F', 'ik': 2, 'nk': 2}


<a name="topic-5"></a>
### 5. read_qe_wfc(filename, orb_list, verbose=0)

Can be used to read in the actual wavefunctions produced

In [33]:
alpha = QE_methods.read_qe_wfc("wd/res/x0.export/wfc.1", [1,2,3,4], verbose=0)
beta = QE_methods.read_qe_wfc("wd/res/x0.export/wfc.2", [1,2,3,4], verbose=0)

path= Kpoint.1
path= Kpoint.1
path= Kpoint.2
path= Kpoint.2


In [34]:
print(alpha)
print(alpha.num_of_rows, alpha.num_of_cols)

print(beta)
print(beta.num_of_rows, beta.num_of_cols)

36889 4
36889 4


Orthogonality and normalization

Below we can see that MO overlaps <alpha(i)|alpha(j)> are almost orthonormal - the diagonal elements are coorectly 1.0

But the off-diagonal elements are not quite 0.0

Same is true for <beta(i)|beta(j)> 

However, there is no any expectation about the orthogonality or normalization across the two sets

In [35]:
S_aa = alpha.H() * alpha
S_bb = beta.H() * beta
S_ab = alpha.H() * beta

In [36]:
def print_mat(X):
    nr, nc = X.num_of_rows, X.num_of_cols
    for i in range(nr):
    
        line = ""
        for j in range(nc):
        
            line = line + "%8.5f  " % (X.get(i,j).real)
        print(line)

print("S_aa")
print_mat(S_aa)

print("S_bb")
print_mat(S_bb)

print("S_ab")
print_mat(S_ab)

S_aa
 1.00000  -0.08150  -0.04761   0.03307  
-0.08150   1.00000   0.03712  -0.02467  
-0.04761   0.03712   1.00000  -0.01177  
 0.03307  -0.02467  -0.01177   1.00000  
S_bb
 1.00000  -0.08093  -0.04785   0.03336  
-0.08093   1.00000   0.03755  -0.02485  
-0.04785   0.03755   1.00000  -0.01246  
 0.03336  -0.02485  -0.01246   1.00000  
S_ab
 0.17301  -0.01049  -0.00412   0.00214  
-0.01635   0.15735   0.00452  -0.00236  
-0.00102  -0.00083  -0.06732   0.00139  
 0.01173  -0.00747  -0.00248   0.24420  


<a name="topic-6"></a>
### 6. QE_utils.orthogonalize_orbitals(C)
Can be used to orthogonalize orbitals if they are not.

So lets transform alpha and beta orbitals such they are now orthonormal within each set.

The resulting orbitals are not orthonormal across the two sets still  

In [37]:
alp = QE_utils.orthogonalize_orbitals(alpha)
bet = QE_utils.orthogonalize_orbitals(beta)

S_aa = alp.H() * alp
S_bb = bet.H() * bet
S_ab = alp.H() * bet

print("S_aa")
print_mat(S_aa)

print("S_bb")
print_mat(S_bb)

print("S_ab")
print_mat(S_ab)


Testing if S is invertabile

[4, 1]
Det =  (0.9881907434095535-8.072456740942809e-20j)

Testing if S is invertabile

[4, 1]
Det =  (0.9883023952546621+8.762301957458487e-20j)
S_aa
 1.00000   0.00000   0.00000  -0.00000  
 0.00000   1.00000  -0.00000   0.00000  
 0.00000  -0.00000   1.00000   0.00000  
-0.00000   0.00000   0.00000   1.00000  
S_bb
 1.00000  -0.00000   0.00000   0.00000  
-0.00000   1.00000  -0.00000   0.00000  
 0.00000  -0.00000   1.00000  -0.00000  
 0.00000   0.00000  -0.00000   1.00000  
S_ab
 0.17301   0.00000  -0.00001   0.00000  
-0.00001   0.15735   0.00024  -0.00006  
-0.00001   0.00005  -0.06733   0.00028  
 0.00002   0.00017   0.00083   0.24420  


<a name="topic-7"></a>
### 7. QE_utils.merge_orbitals(Ca, Cb)

Sometimes (usually in the non-collinear case), we want to have a single set of orbitals (many are nearly doubly degenerate), not just alpha and beta components. We can prepare the single set from the spinor components using this function. In this example, we just gonna mimic non-collinear SOC calculations, pretending that alpha and beta orbital sets are the spinor components. 


In [38]:
C = QE_utils.merge_orbitals(alpha, beta)

S = C.H() * C
print_mat(S)

 1.00000  -0.08150  -0.04761   0.03307   0.17301  -0.01049  -0.00412   0.00214  
-0.08150   1.00000   0.03712  -0.02467  -0.01635   0.15735   0.00452  -0.00236  
-0.04761   0.03712   1.00000  -0.01177  -0.00102  -0.00083  -0.06732   0.00139  
 0.03307  -0.02467  -0.01177   1.00000   0.01173  -0.00747  -0.00248   0.24420  
 0.17301  -0.01635  -0.00102   0.01173   1.00000  -0.08093  -0.04785   0.03336  
-0.01049   0.15735  -0.00083  -0.00747  -0.08093   1.00000   0.03755  -0.02485  
-0.00412   0.00452  -0.06732  -0.00248  -0.04785   0.03755   1.00000  -0.01246  
 0.00214  -0.00236   0.00139   0.24420   0.03336  -0.02485  -0.01246   1.00000  


<a name="topic-8"></a>
### 8. QE_utils.orthogonalize_orbitals2(Ca, Cb)

This is a special orthogonalization procedure - the one for 2-component spinors. The inputs are assumed to be the components for each orbital. The orthogonalization works such that it is S_aa + S_bb = I

In [39]:
alpha = QE_methods.read_qe_wfc("wd/res/x0.export/wfc.1", [1,2,3,4], verbose=0)
beta = QE_methods.read_qe_wfc("wd/res/x0.export/wfc.2", [1,2,3,4], verbose=0)

path= Kpoint.1
path= Kpoint.1
path= Kpoint.2
path= Kpoint.2


In [40]:
alp, bet = QE_utils.orthogonalize_orbitals2(alpha, beta)

S_aa = alp.H() * alp
S_bb = bet.H() * bet
print("S_aa")
print_mat(S_aa)

print("S_bb")
print_mat(S_bb)

print("S_aa + S_bb")
print_mat(S_aa + S_bb)

S_ab = alp.H() * bet
print("S_ab")
print_mat(S_ab)


S_aa
 0.49999  -0.00014   0.00006  -0.00007  
-0.00014   0.49999  -0.00010   0.00004  
 0.00006  -0.00010   0.50000   0.00016  
-0.00007   0.00004   0.00016   0.50000  
S_bb
 0.50001   0.00014  -0.00006   0.00007  
 0.00014   0.50001   0.00010  -0.00004  
-0.00006   0.00010   0.50000  -0.00016  
 0.00007  -0.00004  -0.00016   0.50000  
S_aa + S_bb
 1.00000  -0.00000   0.00000  -0.00000  
-0.00000   1.00000  -0.00000   0.00000  
 0.00000  -0.00000   1.00000   0.00000  
-0.00000   0.00000   0.00000   1.00000  
S_ab
 0.08657  -0.00003   0.00001   0.00004  
-0.00005   0.07869   0.00008  -0.00006  
 0.00003  -0.00004  -0.03377   0.00018  
 0.00004   0.00006   0.00040   0.12212  


<a name="topic-9"></a>
### 9. read_qe_wfc_grid(filename, verbose=0)

Can be used to read the grid points for the given PW representation.

In [41]:
G1 = QE_methods.read_qe_wfc_grid("wd/res/x0.export/grid.1", verbose=0)

path= Kpoint.1


In [42]:
print(len(G1))

36889


In [43]:
for i in range(10):
    print(F"{i} \t {G1[i].x} \t {G1[i].y} \t {G1[i].z}")

0 	 0.0 	 0.0 	 0.0
1 	 -1.0 	 0.0 	 0.0
2 	 1.0 	 0.0 	 0.0
3 	 -2.0 	 0.0 	 0.0
4 	 2.0 	 0.0 	 0.0
5 	 0.0 	 -1.0 	 0.0
6 	 0.0 	 0.0 	 -1.0
7 	 0.0 	 0.0 	 1.0
8 	 0.0 	 1.0 	 0.0
9 	 -1.0 	 -1.0 	 0.0
